<a href="https://colab.research.google.com/github/NneamakaKateOkafor/hello-world/blob/master/Twitter_US_Airline_Sentiment_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Twitter US Airline Sentiment Project

**Data Description:**

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from
February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed
by categorizing negative reasons (such as "late flight" or "rude service").
bold text

**1. Import the libraries, load dataset, print shape of data, data description. (5 Marks)**


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install contractions

In [ ]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re, string, unicodedata
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import contractions
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer, LancasterStemmer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

In [ ]:
# Define the pathway to the folder where the csv file is stored
project_path = '/content/drive/My Drive/Colab Notebooks/'

dataset_file = project_path + 'Tweets.csv'

In [ ]:
# Read the csv file
mydata = pd.read_csv(dataset_file)

In [ ]:
# Basic Information of the dataset including data types
mydata.info()

In [ ]:
# Check only the data types not the whole information
mydata.dtypes

In [ ]:
# Analysis of the body of distributions / head
mydata.head()

In [ ]:
# Display the shape of the data
mydata.shape

In [ ]:
# Describe the attributes of this dataset (name, range of values observed, mean and median, standard deviation and quartiles)
mydata.describe().transpose()

In [ ]:
# Number of unique values in each column
mydata.nunique()

In [ ]:
# Check for duplicate rows
mydata.duplicated().sum()

In [ ]:
# To remove duplicates, we can use the following code, but i will not remove them since all sentiments in this project are required.
# mydata = mydata.drop_duplicates()

In [ ]:
# Skewness of the dataset
mydata.skew()

In [ ]:
# Null values
mydata.isnull().sum(axis=0)

**2. Understand of data-columns: (5 Marks)**

a. Drop all other columns except “text” and “airline_sentiment”


In [ ]:
# Create a copy of mydata
mydata2 = mydata.copy()
mydata2.drop(['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence',
              'airline','airline_sentiment_gold','name',
              'negativereason_gold','retweet_count','tweet_coord','tweet_created',
              'tweet_location','user_timezone'], axis=1, inplace=True)

mydata2

b. Check the shape of data

In [ ]:
# Display the shape of the data after dropping the columns
mydata2.shape

**Note: The columns reduced from 15 to 2**


c. Print first 5 rows of data

In [ ]:
# Display full dataframe information (Non-truncated Text column)
pd.set_option('display.max_colwidth', None)
mydata2.head

**3. Text pre-processing: Data preparation. (20 Marks)**

a. Html tag removal.

b. Tokenization.

c. Remove the numbers.

d. Removal of Special Characters and Punctuations.

e. Conversion to lowercase.

f. Lemmatize or stemming.

g. Join the words in the list to convert back to text string in the dataframe. (So that each row
contains the data in text format.)

h. Print first 5 rows of data after pre-processing.


In [ ]:
# HTML tag removal
def remove_html_tags(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

mydata2['text'] = mydata2['text'].apply(lambda x: remove_html_tags(x))

In [ ]:
mydata2

In [ ]:
# Remove the numbers
def remove_numbers(text):
  text = re.sub(r'\d+', '', str(text))
  return text


mydata2['text'] = mydata2['text'].apply(lambda x: remove_numbers(x))

In [ ]:
mydata2.head()

In [ ]:
# Removal of Special Characters and Punctuations
def remove_punts_characters(text):
  new_text = []
  clean_txt = re.sub(r'[#,@,&,?,!]', '', text)
  clean_txt = contractions.fix(clean_txt)
  #new_txt = re.sub(r'[^\w\s]', ',', clean_txt)
  #if new_txt != '':
    #new_text.append(new_txt)
  
  return clean_txt

mydata2['text'] = mydata2['text'].apply(lambda x: remove_punts_characters(x))

In [ ]:
mydata2

In [ ]:
# Tokenization
mydata2['text'] = mydata2.apply(lambda x: nltk.word_tokenize(x['text']), axis=1)

In [ ]:
mydata2

In [ ]:
stopwords = stopwords.words('english')

In [ ]:
stopwords

In [ ]:
# Conversion to lowercase
def to_lowercase(text):
  new_text = []
  for txt in text:
    new_txt = txt.lower()
    new_text.append(new_txt)
  return new_text

In [ ]:
# Remove non-ASCII
def remove_non_ascii(text):
  """Remove non-ASCII characters from list of tokenized words"""
  new_text = []
  for txt in text:
    new_txt = unicodedata.normalize('NFKD',txt).encode('ascii', 'ignore').decode('utf-8','ignore')
    new_text.append(new_txt)
  return new_text

In [ ]:
# Remove Punctuation
def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    new_text = []
    for txt in text:
        new_txt = re.sub(r'[^\w\s]', '', text)
        if new_txt != '':
            new_text.append(new_txt)
    return new_text

In [ ]:
# Remove Stopwords
def remove_stopwords(text):
    """Remove stop words from list of tokenized words"""
    new_text = []
    for txt in text:
        if txt not in stopwords:
          new_text.append(txt)
    return new_text

In [ ]:
# Lemmatize or stemming
def lemmatize_list(text):
   lemmatizer = WordNetLemmatizer()
   new_text = []
   for txt in text:
     new_text.append(lemmatizer.lemmatize(txt))
   return new_text

def stemming_list(text):
   new_text = []
   for txt in text:
     new_text.append(LancasterStemmer.stem(txt))

   return new_text 


In [143]:
# Join the words in the list to convert back to text string in the dataframe
def normalize(text):
  text = remove_non_ascii(text)
  text = to_lowercase(text)
 # text = remove_punctuation(text)
  text = remove_stopwords(text)
  text = lemmatize_list(text)
  return '.'.join(text)

mydata2['text'] = mydata2.apply(lambda x: normalize(x['text']), axis=1)
mydata2

,airline_sentiment,text
0,neutral,virginamerica.dhepburn.said..
1,positive,virginamerica.plus.added.commercial.experience.....tacky..
2,neutral,virginamerica.today.....must.mean.need.take.another.trip
3,negative,virginamerica.really.aggressive.blast.obnoxious.``.entertainment.''.guest.'.face.little.recourse
4,negative,virginamerica.really.big.bad.thing
...,...,...
14635,positive,americanair.thank.got.different.flight.chicago..
14636,negative,americanair.leaving.minute.late.flight...warning.communication.minute.late.flight...called.shitty.customer.svc
14637,neutral,americanair.please.bring.american.airline.blackberry
14638,negative,americanair.money.change.flight.answer.phone.suggestion.make.commitment


In [144]:
# Print first 5 rows of data after pre-processing
mydata2.head()

,airline_sentiment,text
0,neutral,virginamerica.dhepburn.said..
1,positive,virginamerica.plus.added.commercial.experience.....tacky..
2,neutral,virginamerica.today.....must.mean.need.take.another.trip
3,negative,virginamerica.really.aggressive.blast.obnoxious.``.entertainment.''.guest.'.face.little.recourse
4,negative,virginamerica.really.big.bad.thing


**4. Vectorization: (10 Marks)**

a. Use CountVectorizer.

b. Use TfidfVectorizer.

In [145]:
# Use Count Vectorizer
count_vectorizer = CountVectorizer(max_features=1000)
count_vect_data_features = count_vectorizer.fit_transform(mydata2['text'])
count_vect_data_features = count_vect_data_features.toarray() 
count_vect_data_features.shape

(14640, 1000)

In [146]:
# Use Tfidf Vectorizer
tfidf_vectoriser = TfidfVectorizer(max_features=2000)
tfidf_data_features = tfidf_vectoriser.fit_transform(mydata2['text'])
tfidf_data_features = tfidf_data_features.toarray()
tfidf_data_features.shape

(14640, 2000)

**5. Fit and evaluate model using both type of vectorization. (6+6 Marks)**

In [147]:
labels = mydata2['airline_sentiment'].apply(lambda x:0 if x=='negative' else 1 )
labels.dtype

dtype('int64')

In [148]:
 Classifiers = [
    LogisticRegression(C=0.000000001,solver='liblinear',max_iter=200),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier()]
   

In [149]:
# Fit and Evaluate Model using Count Vectorizer
Accuracy=[]
Model=[]
features=[]

def model_fit(Xtrain,ytrain,xtest,ytest,feature_type):
 

  #Accuracy=[]
  #Model=[]
  for clf in Classifiers:

    try:
      fit = clf.fit(Xtrain,ytrain)
      pred = fit.predict(xtest)
    except Exception:
      fit = clf.fit(Xtrain,ytrain)
      pred = fit.predict(xtest)
    accuracy = accuracy_score(pred,ytest)
    Accuracy.append(accuracy)
    Model.append(clf.__class__.__name__)
    features.append(feature_type)
    print('Accuracy of '+clf.__class__.__name__+' is '+str(accuracy)+ ':' + feature_type) 
    print(classification_report(pred,ytest))
    
    #Confusion Matrix
    #cm = confusion_matrix(pred,ytest)
    #plt.figure()
    #plot_confusion_matrix(cm,cmap=plt.cm.Blues)
   
    #plot_confusion_matrix((clf,pred, ytest)
    #plt.xticks(range(2), ['Negative', 'Positive'],color='black')
    #plt.yticks(range(2), ['Negative', 'Positive'])
    #plt.xlabel('Predicted Label')
    #plt.ylabel('True Label')
    #plt.show()


In [150]:
X_train, X_test, y_train, y_test = train_test_split(count_vect_data_features,labels, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(10248, 1000)
(4392, 1000)
(10248,)
(4392,)


In [151]:
model_fit(X_train,y_train,X_test,y_test,'CountVectorizer')


Accuracy of LogisticRegression is 0.6407103825136612:CountVectorizer
              precision    recall  f1-score   support

           0       1.00      0.64      0.78      4392
           1       0.00      0.00      0.00         0

    accuracy                           0.64      4392
   macro avg       0.50      0.32      0.39      4392
weighted avg       1.00      0.64      0.78      4392



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy of DecisionTreeClassifier is 0.7408925318761385:CountVectorizer
              precision    recall  f1-score   support

           0       0.78      0.81      0.80      2740
           1       0.66      0.63      0.65      1652

    accuracy                           0.74      4392
   macro avg       0.72      0.72      0.72      4392
weighted avg       0.74      0.74      0.74      4392

Accuracy of RandomForestClassifier is 0.7975865209471766:CountVectorizer
              precision    recall  f1-score   support

           0       0.85      0.84      0.84      2855
           1       0.71      0.72      0.71      1537

    accuracy                           0.80      4392
   macro avg       0.78      0.78      0.78      4392
weighted avg       0.80      0.80      0.80      4392

Accuracy of AdaBoostClassifier is 0.7786885245901639:CountVectorizer
              precision    recall  f1-score   support

           0       0.87      0.80      0.83      3028
           1       0.6

In [152]:
# Fit and Evaluate Model using Tfidf Vectorizer
X_train, X_test, y_train, y_test = train_test_split(tfidf_data_features,labels, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(10248, 2000)
(4392, 2000)
(10248,)
(4392,)


In [153]:
model_fit(X_train,y_train,X_test,y_test,'TfidfVectorizer');

Accuracy of LogisticRegression is 0.6407103825136612:TfidfVectorizer
              precision    recall  f1-score   support

           0       1.00      0.64      0.78      4392
           1       0.00      0.00      0.00         0

    accuracy                           0.64      4392
   macro avg       0.50      0.32      0.39      4392
weighted avg       1.00      0.64      0.78      4392



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy of DecisionTreeClassifier is 0.735655737704918:TfidfVectorizer
              precision    recall  f1-score   support

           0       0.78      0.80      0.79      2759
           1       0.65      0.63      0.64      1633

    accuracy                           0.74      4392
   macro avg       0.72      0.71      0.72      4392
weighted avg       0.73      0.74      0.73      4392

Accuracy of RandomForestClassifier is 0.8089708561020036:TfidfVectorizer
              precision    recall  f1-score   support

           0       0.90      0.82      0.86      3093
           1       0.65      0.78      0.71      1299

    accuracy                           0.81      4392
   macro avg       0.77      0.80      0.78      4392
weighted avg       0.83      0.81      0.81      4392

Accuracy of AdaBoostClassifier is 0.7773224043715847:TfidfVectorizer
              precision    recall  f1-score   support

           0       0.89      0.79      0.84      3152
           1       0.58

In [ ]:
myresult = pd.DataFrame({'Model': Model, 'Accuracy': Accuracy, 'Vectorizer': features})
myresult = myresult[['Model', 'Accuracy', 'Vectorizer']]

In [ ]:
myresult

The RandomForestClassifier has the highest accuracy and F1 scores

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_data_features,labels, test_size=0.3, random_state=42)
X_train.shape
y_train.shape

In [ ]:
forest = RandomForestClassifier(n_estimators=200, n_jobs=4)
forest = forest.fit(X_train,y_train)

In [ ]:
results = forest.predict(X_test)
accuracy = accuracy_score(results,y_test)
print(accuracy)
#print(np.mean(cross_val_score(forest, tfidf_data_features, cv=10)))

**6. Summarize your understanding of the application of Various Pre-processing and Vectorization and
performance of your model on this dataset. (8 Marks)**

The dataset reviews were converted to integers (0,1)

I pre-processed the text column by doing the following:
*   All HTML tags were removed using BeautifulSoup package because the data was acquired from the web
*   The numbers and special characters were removed and replaced with spaces with the help of regular expression 
*   The contraction package converted words, reduced and/or combined them by dropping the apostrophe to the their full words
*   **Single words were Tokenized after the removal of special characters and punctuations (instead of before) to avoid ending up with gibberish (combination of syllables with no meaning) after joining the words and converting back to text string in the dataframe**
*   Stopwords from the tokenized text were removed and subsequently, converted to lowercase
*   The tokenized words were Lemmatized
 
Because models can only be run on integers, CountVectorizer and TfidfVectorizer were deployed to convert words to numbers.

This model was built and evaluated using the following:  
*   LogisticRegression
*   DecisionTreeClassifier
*   RandomForestClassifier
*   AdaBoostClassifier

The RadomForestClassifier produced the highest accuracy of approximately 81% using the TfidfVectorizer. 

To increase the accuracy of this model, i tuned the hyperparameters of the RandomForest Classifier model and increased the number of features from 1000 to 2000.
